In [1]:
# Importación de librerias
import difflib
import pandas as pd
import numpy as np
import pyodbc

import pandas.io.sql as sqlio
import psycopg2
import json

# Importar tabla de datalab a dataframe

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=10.12.6.38;'
                      'Database=AppCatalogo;'
                      'UID=usrappcatalogosqas;'
                      'PWD=Belcorp2015;')
script ="""
select
--top 10 
CodPais
,CodMarca
,CodCampana
,CUV
,CodSAP
,CUVPadre
,NumPaginaCatalogo
,DescripcionComercial
,NombreComercial
,Descripcion
,PrecioValorizado
,PrecioCatalogo
,Volumen
,Imagen
FROM [dbo].[ProductoCampana]
where codcampana BETWEEN '202005' AND '202006' and NumPaginaCatalogo <> 0 and imagen is not NULL;
""" 

cursor = conn.cursor()

script_output = pd.read_sql(script, conn)
df_productos = pd.DataFrame(script_output)

conn.commit()
conn.close()

print(df_productos.shape)
df_productos.head(3)

(10, 14)


,CodPais,CodMarca,CodCampana,CUV,CodSAP,CUVPadre,NumPaginaCatalogo,DescripcionComercial,NombreComercial,Descripcion,PrecioValorizado,PrecioCatalogo,Volumen,Imagen
0,BO,C,202005,03385,200094220,03115,36,CZ STUDIO LOOK LL APPLE POP,Studio Look Labial Líquido. Apple pop,Labial líquido de color intenso.,83.52,55.61,6 ml / 0.2 fl. oz.,BO_200094220.jpg
1,BO,E,202005,29207,SE0002473,,56,ES TIARE BLOSSOM EDP 50ML,Set Tiaré Blossom,"Incluye: Tiaré Blossom, Máscara Mega Full Size...",487.20,205.32,"Per: 50 ml, Más: 8 g, Lab: 4 g.",BO_SE0002473.jpg
2,BO,E,202005,05021,200095440,05021,23,ES PRO MASC FULLSIZE F-R AZUL,Máscara Mega Full Size fácil retirado. Azul Fá...,Máscara para pestañas visiblemente más largas ...,125.28,83.45,8 g / .28 oz.,BO_200095440.jpg


In [2]:
CodPais_orden = {'CodPais': ['MX', 'PE', 'CL', 'CO', 'EC', 'DO', 'CR', 'GT', 'BO', 'SV' , 'PR', 'PA'],
'CodPais_Orden': [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]}

CodPais_orden_df = pd.DataFrame(CodPais_orden)
CodPais_orden_df

,CodPais,CodPais_Orden
0,MX,12
1,PE,11
2,CL,10
3,CO,9
4,EC,8
5,DO,7
6,CR,6
7,GT,5
8,BO,4
9,SV,3


In [3]:
#result = pd.concat([df_productos, CodPais_order_df], axis=1, join='inner', ignore_index=False, keys='CodPais', levels=None)
df_productos_orden= pd.merge(df_productos, CodPais_orden_df, how='inner', on='CodPais',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

df_productos_orden['Rank'] = df_productos_orden.sort_values(['CodCampana','CodPais_Orden'], ascending=[False,False]) \
             .groupby(['CodSAP']) \
             .cumcount() + 1

print(df_productos_orden.shape)
df_productos_orden.head(3)

(10, 16)


,CodPais,CodMarca,CodCampana,CUV,CodSAP,CUVPadre,NumPaginaCatalogo,DescripcionComercial,NombreComercial,Descripcion,PrecioValorizado,PrecioCatalogo,Volumen,Imagen,CodPais_Orden,Rank
0,BO,C,202005,03385,200094220,03115,36,CZ STUDIO LOOK LL APPLE POP,Studio Look Labial Líquido. Apple pop,Labial líquido de color intenso.,83.52,55.61,6 ml / 0.2 fl. oz.,BO_200094220.jpg,4,1
1,BO,E,202005,29207,SE0002473,,56,ES TIARE BLOSSOM EDP 50ML,Set Tiaré Blossom,"Incluye: Tiaré Blossom, Máscara Mega Full Size...",487.20,205.32,"Per: 50 ml, Más: 8 g, Lab: 4 g.",BO_SE0002473.jpg,4,1
2,BO,E,202005,05021,200095440,05021,23,ES PRO MASC FULLSIZE F-R AZUL,Máscara Mega Full Size fácil retirado. Azul Fá...,Máscara para pestañas visiblemente más largas ...,125.28,83.45,8 g / .28 oz.,BO_200095440.jpg,4,1


In [4]:
#Prioridad según pais
df_productos_orden_f = df_productos_orden[df_productos_orden.Rank == 1]

df_productos_orden_f = df_productos_orden_f.reset_index()

print(df_productos_orden_f.shape)
df_productos_orden_f.head(3)

(10, 17)


,index,CodPais,CodMarca,CodCampana,CUV,CodSAP,CUVPadre,NumPaginaCatalogo,DescripcionComercial,NombreComercial,Descripcion,PrecioValorizado,PrecioCatalogo,Volumen,Imagen,CodPais_Orden,Rank
0,0,BO,C,202005,03385,200094220,03115,36,CZ STUDIO LOOK LL APPLE POP,Studio Look Labial Líquido. Apple pop,Labial líquido de color intenso.,83.52,55.61,6 ml / 0.2 fl. oz.,BO_200094220.jpg,4,1
1,1,BO,E,202005,29207,SE0002473,,56,ES TIARE BLOSSOM EDP 50ML,Set Tiaré Blossom,"Incluye: Tiaré Blossom, Máscara Mega Full Size...",487.20,205.32,"Per: 50 ml, Más: 8 g, Lab: 4 g.",BO_SE0002473.jpg,4,1
2,2,BO,E,202005,05021,200095440,05021,23,ES PRO MASC FULLSIZE F-R AZUL,Máscara Mega Full Size fácil retirado. Azul Fá...,Máscara para pestañas visiblemente más largas ...,125.28,83.45,8 g / .28 oz.,BO_200095440.jpg,4,1


In [27]:
#AGREGAR COLUMNA "URL_AUX" AL DATAFRAME
df_productos_orden_f["URL_aux"] = ''
i=0
for index_img, row_img in df_productos_orden_f.iterrows():
    i=i+1
    df_productos_orden_f["URL_aux"].loc[index_img] = "https://s3.amazonaws.com/catalogodigitalbelcorp/"+row_img['CodPais']+"/"+row_img['CodCampana']+"/"+row_img['CodMarca']+"/productos/"+row_img['CodPais']+"_"+row_img['CodSAP']+".jpg"
    

In [140]:
import requests

for index_img, row_img in df_productos_orden_f.iterrows():
    
    URL_aux = "https://s3.amazonaws.com/catalogodigitalbelcorp/"+row_img['CodPais']+"/"+row_img['CodCampana']+"/"+row_img['CodMarca']+"/productos/"+row_img['CodPais']+"_"+row_img['CodSAP']+".jpg"
    
    with open(r"d:\Users\rcocchella\Desktop\Imagenes\img_"+row_img['CodSAP']+".jpg", "wb") as f:
        f.write(requests.get(URL_aux).content)
        f.close()


In [193]:
import PIL
from PIL import Image

#image = PIL.Image.open(r"d:\Users\rcocchella\Desktop\Imagenes\img_SL0005783.jpg")

#width, height = image.size

#print(width, height)

df_productos_orden_f["img_width"] = 0
df_productos_orden_f["img_heigth"] = 0


for index_img, row_img in df_productos_orden_f.iterrows():
    
    image = PIL.Image.open(r"d:\Users\rcocchella\Desktop\Imagenes\img_"+row_img['CodSAP']+".jpg")
    
    width, height = image.size
    
    df_productos_orden_f["img_width"].iloc[index_img] = width
    df_productos_orden_f["img_heigth"].iloc[index_img] = height
    
        
    #with open(r"d:\Users\rcocchella\Desktop\Imagenes\img_"+row_img['CodSAP']+".jpg", "wb") as f:
    #    f.write(requests.get(URL_aux).content)
    #    f.close()

In [202]:
print(df_productos_orden_f.groupby('img_width')['CodSAP'].nunique())

df_productos_orden_f.groupby('img_heigth')['CodSAP'].nunique()

img_width
600    2759
800       2
Name: CodSAP, dtype: int64


img_heigth
600    2759
800       2
Name: CodSAP, dtype: int64